In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

### Defining Setup Time and Hourly Yield Constants

In [2]:
T = pd.read_csv('setup_constants.csv')
R = pd.read_csv('yield_constants.csv')

In [3]:
T.index += 1
R.index += 1

### Defining Weekly Demand Constants

In [4]:
D = pd.read_csv('weekly_demands.csv')

In [5]:
D.index += 1

# Defining Problem

In [6]:
model = gp.Model('base_problem')

Restricted license - for non-production use only - expires 2026-11-23


## Variables

### Adding Binary Setup Variables

In [7]:
Sw = []
for w in range(2):
    S = []
    for i in range(5):
        S_i = []
        for j in range(5):
            S_i.append(model.addVar(vtype=GRB.BINARY, name=f"S_{i}_{j}_{w}", lb=0))
        S.append(S_i)
    Sw.append(S)

### Adding Binary Overtime Setup Variables

In [8]:
Spw = []
for w in range(2):
    Sp = []
    for i in range(5):
        Sp_i = []
        for j in range(5):
            Sp_i.append(model.addVar(vtype=GRB.BINARY, name=f"Sp_{i}_{j}_{w}", lb=0))
        Sp.append(Sp_i)
    Spw.append(Sp)

### Adding Production Variables

In [9]:
Pw = []
for w in range(2):
    P = []
    for i in range(5):
        P_i = []
        for j in range(5):
            P_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"P_{i}_{j}_{w}", lb=0))
        P.append(P_i)
    Pw.append(P)

### Adding Overtime Production Variables

In [10]:
Ppw = []
for w in range(2):
    Pp = []
    for i in range(5):
        Pp_i = []
        for j in range(5):
            Pp_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"Pp_{i}_{j}_{w}", lb=0))
        Pp.append(Pp_i)
    Ppw.append(Pp)

### Adding Use Variables

In [11]:
Uw = []
for w in range(2):
    U = []
    for i in range(5):
        U_i = []
        for j in range(5):
            U_i.append(model.addVar(vtype=GRB.BINARY, name=f"U_{i}_{j}_{w}", lb=0))
        U.append(U_i)
    Uw.append(U)

### Adding Overtime Use Variables

In [12]:
Upw = []
for w in range(2):
    Up = []
    for i in range(5):
        Up_i = []
        for j in range(5):
            Up_i.append(model.addVar(vtype=GRB.BINARY, name=f"Up_{i}_{j}_{w}", lb=0))
        Up.append(U_i)
    Upw.append(Up)

In [13]:
Tw = []
for w in range(2):
    Tw.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"T_{w}", lb=0))

In [14]:
inventory = []  
for w in range(2):  
    inventory_w = []  
    for i in range(5): 
        # Add a continuous variable for inventory of each part for the current week
        inventory_w.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"Inventory_{i}_{w}", lb=0))
        inventory_w[i].start = 0
    inventory.append(inventory_w)  # Add the list of part inventory for this week to overall list of inventory


In [15]:
LoD = []  
for w in range(2):
    LoD_w = []
    for i in range(5):
        # Add a continuous variable for unmet demand of each part for the current week
        LoD_w.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"LoD_{i}_{w}", lb=0))
        LoD_w[i].start = 0
    LoD.append(LoD_w)  # Add the list of unmet demand for this week to overall list of unmet demand


## Constraints

### Demand Constraints

In [16]:
for w in range(2):
    for i in range(5):
        constr_expr = gp.quicksum(
            Pw[w][j][i] * R.iloc[j, i] + Ppw[w][j][i] * R.iloc[j, i] for j in range(5)
        )

        if w == 0:  # First Week
            model.addConstr(
                constr_expr + LoD[w][i] >= D.iloc[10, i] + inventory[w][i],
                name=f'Demand_Prod_{i+1}_{w}_Condition1'
            )
        else:  # Subsequent Week
            model.addConstr(
                constr_expr + inventory[w-1][i] + LoD[w][i] >= D.iloc[11, i],
                name=f'Demand_Prod_{i+1}_{w}_Condition2'
            )



### Normal Production Time Constraints

In [17]:
for w in range(2):
    for i in range(5):
        constr_expr = gp.quicksum(Sw[w][i][j]*T.iloc[i,j] + Pw[w][i][j] for j in range(5))
        model.addConstr(constr_expr <= 120, name=f'Prod_Hrs_Mach_{i+1}_{w}')

### Overtime Production Time Constraints

In [18]:
for w in range(2):
    for i in range(5):
        constr_expr = gp.quicksum(Spw[w][i][j]*T.iloc[i,j] + Ppw[w][i][j] for j in range(5))
        model.addConstr(constr_expr <= 48, name=f'OT_Prod_Hrs_Mach_{i+1}_{w}')

### Production Use Linkage Constraints
- Regular Time and Overtime are both included here

In [19]:
for w in range(2):
    for i in range(5):
        for j in range(5):
            model.addConstr(Pw[w][i][j]*R.iloc[i,j] <= 10000*Uw[w][i][j], name=f'UseProdLinkage_Mach_{i+1}_Prod_{j+1}_Week_{w}')
            model.addConstr(Ppw[w][i][j]*R.iloc[i,j] <= 10000*(Uw[w][i][j]+Upw[w][i][j]), name=f'OTUseProdLinkage_Mach_{i+1}_Prod_{j+1}_Week_{w}')

### Setup Use Linkage Constraints
- Regular Time and Overtime are both included here

In [20]:
for i in range(5):
    for j in range(5):
        model.addConstr(Uw[0][i][j] <= 10*Sw[0][i][j], name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_0')
        model.addConstr(Upw[0][i][j] <= 10*(Sw[0][i][j]+Spw[0][i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_0')

for i in range(5):
    for j in range(5):
        model.addConstr(Uw[1][i][j] <= 10*(Sw[1][i][j]+Sw[0][i][j]+Spw[0][i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_1')
        model.addConstr(Upw[1][i][j] <= 10*(Sw[0][i][j]+Spw[0][i][j]+Sw[1][i][j]+Spw[1][i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_1')

### Maximisation Constraints
- Since we're trying to convert the max(overtime[i]) function - which is not a linear function - to a linear function for the purpose of solving it as an LP, we add additional constraints.

In [21]:
for w in range(2):
    for i in range(5):
        model.addConstr(
            gp.quicksum(
                Spw[w][i][j] * T.iloc[i, j] + Ppw[w][i][j] for j in range(5)
            ) <= Tw[w],
            name=f'MaxConstr_{i}_{w}'
        )


## Objective Function

In [22]:
obj_parts = []
for w in range(2):
    for i in range(5):
        obj_parts.append(gp.quicksum(Spw[w][i][j]*T.iloc[i,j] + Ppw[w][i][j] for j in range(5)))

In [23]:
model.setObjective(
    gp.quicksum(Tw[w] * 40 for w in range(len(Tw))) +  # Summing over Tw
    gp.quicksum(obj_parts) * 30 +  # Summing obj_parts (already a summation)
    gp.quicksum(LoD[w][i] * 3 for w in range(len(LoD)) for i in range(len(LoD[w]))) +  # Summing LoD
    gp.quicksum(inventory[0][i] * 2 for i in range(5)),  # Summing inventory
    GRB.MINIMIZE
)


In [24]:
model.update()

In [25]:
model.getObjective()

<gurobi.LinExpr: 240.0 Sp_0_0_0 + 240.0 Sp_0_3_0 + 300.0 Sp_1_0_0 + 240.0 Sp_1_1_0 + 300.0 Sp_2_1_0 + 720.0 Sp_2_4_0 + 240.0 Sp_3_1_0 + 360.0 Sp_3_2_0 + 240.0 Sp_4_3_0 + 600.0 Sp_4_4_0 + 240.0 Sp_0_0_1 + 240.0 Sp_0_3_1 + 300.0 Sp_1_0_1 + 240.0 Sp_1_1_1 + 300.0 Sp_2_1_1 + 720.0 Sp_2_4_1 + 240.0 Sp_3_1_1 + 360.0 Sp_3_2_1 + 240.0 Sp_4_3_1 + 600.0 Sp_4_4_1 + 30.0 Pp_0_0_0 + 30.0 Pp_0_1_0 + 30.0 Pp_0_2_0 + 30.0 Pp_0_3_0 + 30.0 Pp_0_4_0 + 30.0 Pp_1_0_0 + 30.0 Pp_1_1_0 + 30.0 Pp_1_2_0 + 30.0 Pp_1_3_0 + 30.0 Pp_1_4_0 + 30.0 Pp_2_0_0 + 30.0 Pp_2_1_0 + 30.0 Pp_2_2_0 + 30.0 Pp_2_3_0 + 30.0 Pp_2_4_0 + 30.0 Pp_3_0_0 + 30.0 Pp_3_1_0 + 30.0 Pp_3_2_0 + 30.0 Pp_3_3_0 + 30.0 Pp_3_4_0 + 30.0 Pp_4_0_0 + 30.0 Pp_4_1_0 + 30.0 Pp_4_2_0 + 30.0 Pp_4_3_0 + 30.0 Pp_4_4_0 + 30.0 Pp_0_0_1 + 30.0 Pp_0_1_1 + 30.0 Pp_0_2_1 + 30.0 Pp_0_3_1 + 30.0 Pp_0_4_1 + 30.0 Pp_1_0_1 + 30.0 Pp_1_1_1 + 30.0 Pp_1_2_1 + 30.0 Pp_1_3_1 + 30.0 Pp_1_4_1 + 30.0 Pp_2_0_1 + 30.0 Pp_2_1_1 + 30.0 Pp_2_2_1 + 30.0 Pp_2_3_1 + 30.0 Pp_2_4_1 + 30.

In [26]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 240 rows, 322 columns and 815 nonzeros
Model fingerprint: 0x10918c13
Variable types: 122 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 5e+03]

User MIP start did not produce a new incumbent solution

Found heuristic solution: objective 117600.00000
Presolve removed 142 rows and 224 columns
Presolve time: 0.00s
Presolved: 98 rows, 98 columns, 329 nonzeros
Variable types: 57 continuous, 41 integer (41 binary)

Root relaxation: objective 1.359547e+04, 98 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

In [27]:
assert model.Status == GRB.OPTIMAL

In [28]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sw[0][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 8 0 0 8 0 
Machine 2: 10 0 0 0 0 
Machine 3: 0 10 0 0 24 
Machine 4: 0 8 12 0 0 
Machine 5: 0 0 0 0 20 


In [29]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sw[1][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 8 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [30]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Spw[0][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 8 0 


In [31]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Spw[1][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [32]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pw[0][i][j].X)), end=" ")
    print("")

Machine 1: 1.25 0.0 0.0 102.75 0.0 
Machine 2: 110.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 86.0 0.0 0.0 0.0 
Machine 4: 0.0 0.0 100.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 100.0 


In [33]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pw[1][i][j].X)), end=" ")
    print("")

Machine 1: 46.67752959015096 0.0 0.0 73.32247040984905 0.0 
Machine 2: 89.51139475411318 22.488605245886777 0.0 0.0 0.0 
Machine 3: 0.0 120.0 0.0 0.0 0.0 
Machine 4: 0.0 13.333333333333334 106.66666666666667 0.0 0.0 
Machine 5: 0.0 0.0 0.0 31.800303256613947 88.19969674338606 


In [34]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Ppw[0][i][j].X)), end=" ")
    print("")

Machine 1: 48.0 0.0 0.0 0.0 0.0 
Machine 2: 48.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 34.53133903133903 0.0 0.0 13.468660968660968 
Machine 4: 0.0 14.666666666666666 33.333333333333336 0.0 0.0 
Machine 5: 0.0 0.0 0.0 25.455128205128204 14.544871794871796 


In [35]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Ppw[1][i][j].X)), end=" ")
    print("")

Machine 1: 0.0 0.0 0.0 5.133636589947275 0.0 
Machine 2: 0.0 5.133636589947275 0.0 0.0 0.0 
Machine 3: 0.0 5.133636589947275 0.0 0.0 0.0 
Machine 4: 0.0 5.133636589947275 0.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 5.13363658994728 


In [36]:
print("**** Week 1 ****")
for i in range(5):
    print(f"Part {i+1}:", round(sum([abs(Pw[0][j][i].X+Ppw[0][j][i].X)*R.iloc[j,i] for j in range(5)])), end=" ")
    print("")

**** Week 1 ****
Part 1: 4500 
Part 2: 2271 
Part 3: 5000 
Part 4: 5000 
Part 5: 3800 


In [37]:
print("**** Week 2 ****")
for i in range(5):
    print(f"Part {i+1}:", round(sum([abs(Pw[1][j][i].X+Ppw[1][j][i].X)*R.iloc[j,i] for j in range(5)])), end=" ")
    print("")

**** Week 2 ****
Part 1: 3000 
Part 2: 2800 
Part 3: 4000 
Part 4: 4300 
Part 5: 2800 


In [38]:
print('Final Objective Value: ', model.ObjVal)

Final Objective Value:  15282.089670038698
